# Census 2022 Results for Germany - Accessing the Data API with Python/Pandas
Get your free account at  
https://ergebnisse2011.zensus2022.de/datenbank/online#modal=register  
API Documentation (latest update 2023-11-27)  
German
https://ergebnisse2011.zensus2022.de/datenbank/online/docs/ZENSUS-Webservices_Einfuehrung.pdf  
English
https://ergebnisse2011.zensus2022.de/datenbank/online/docs/ZENSUS-Webservices_Introduction.pdf

In [2]:
import pandas as pd
import requests
import json
# credentials in hidden file
from dotenv import dotenv_values, load_dotenv
# timestamps
import datetime

In [3]:
pd.__version__

'2.1.1'

In [4]:
# convenience function for timestamps while logging
def tStamp():
    return datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

#### Set your download directory

In [5]:
# e.g. relative to current working directory
myDownloads = "download/"

In [6]:
# Load credentials from .env file
load_dotenv()  
usr, pwd = dotenv_values().values()

## URL will likely change with 2022 results

In [7]:
# Set base path for API calls
BASE_URL = 'https://ergebnisse2011.zensus2022.de/api/rest/2020/'

## Check credentials, network and endpoint availability
Status may also return 'exceeded limit of parallel jobs'  
which can happen after too many failures and will suspend the user temporarily

In [8]:
try:
    hello = requests.get(BASE_URL + 'helloworld/logincheck', params={
        'username': usr,
        'password': pwd,
        'language': 'en'
        }, timeout=300)
    
    print(hello.json()["Status"])

except:
    print(tStamp(), usr, hello)

You have been logged in and out successfully!


## Basic table download function
* be kind to the database
* only request the data-depth that you need
* download municipality data to disk, it changes only once a decade
* parallel downloads don't work

In [15]:
# this takes a RESTful/JSON response as input
# usually from the 'data/tablefile' service
# covers up to 2 dimensions, may be extended up to classifyingvariable5

def response2disk(resp, classVar1="", classKey1="", classVar2="", classKey2=""):

    filename = resp.headers["Content-Disposition"].split("=")[1].replace('"','').replace('.zip','')

    # writing the modifications of the table into the filename
    destination = myDownloads+filename+"_"+classVar1+"_"+classKey1+"_"+classVar2+"_"+classKey2+".zip"
    # beware we now have a zip-archive that contains a csv with a much shorter name (without classVar/classKey)

    # in case of asterisk notation convert to sensible filename
    with open(destination.replace("*","x"), 'wb') as f:
        
        f.write(resp.content)

    print(tStamp(), filename, "download complete")

## Method “tablefile“ starts a value retrieval and returns a table
* example function fetches municipal data for selected regions (SH, HH, NI, MV)
* this makes sense if your repeated table requests always cover the same region 
* make sure regionalvariable exists for table in question
* e.g. 2000S (sample) tables are only available for GEOGM3 (pop >10k)
* there can be up to 5 pairs of classifying-variable and -key

In [16]:
def tab2download(tablename, classVar1="", classKey1="", classVar2="", classKey2=""):

    if tablename.find("S") == 4:

        regio = "GEOGM3" # municipalities, pop >10k
    
    else:

        regio = "GEOGM1" # municipalities, 11k for all of Germany
    
    try:

        response = requests.get(BASE_URL + 'data/tablefile', params={
            'username': usr,
            'password': pwd,
            'name': tablename,
            'regionalvariable': regio,       
            'regionalkey': "01*,02*,03*,13*",  # e.g. NDR Sendegebiet
            'classifyingvariable1': classVar1,
            'classifyingkey1': classKey1,           
            'classifyingvariable2': classVar2,
            'classifyingkey2': classKey2,
            #'startyear': 2011,
            'format': "ffcsv",
            'quality': "on", # include quality symbols, see below
            'language': "de",
            'job': "false"   # get the data directly
            }, timeout=600)  # large tables may take some time

        try:
            response2disk(response, classVar1, classKey1, classVar2, classKey2)  # save to disk for re-use
            myFilename = myDownloads+tablename+"_de_flat_"+classVar1+"_"+classKey1+"_"+classVar2+"_"+classKey2+".zip"
            return(tab2df(myFilename.replace("*","x")))
            
        except:
        
            if response.status_code == 200:
                
                # here the api will tell you if your request could not be processed
                # e.g.
                # 'Code': 25, 'Content': 'Mindestens ein Parameter enthält ungültige Werte'
                # 'Code': 90, 'Content': 'Die angeforderte Tabelle ist nicht vorhanden.'
                
                try:
                    print(tStamp()+" : "+tablename+" : "+str(response.json()["Status"])[0:80])
                except:
                    # in case response isn't json formatted
                    print(tStamp()+" : "+tablename+" : "+str(response.text[0:300]))
            
            else:
                # log if api times out or otherwise disconnects (500, 404...)
                print(tStamp()+" : "+tablename+" http code "+str(response.status_code))

    except requests.exceptions.Timeout:

        # log if this request has hit its own timeout limit as set above
        print(tStamp()+" : "+tablename + " timed out")

## Check if table has been downloaded before, then download or load

In [17]:
def tab2reuse(tablename, classVar1="", classKey1="", classVar2="", classKey2=""):

    # check if desired table is already available locally
    myFilename = myDownloads+tablename+"_de_flat_"+classVar1+"_"+classKey1+"_"+classVar2+"_"+classKey2+".zip"
    try:
        myDF = tab2df(myFilename.replace("*","x"))
    except:
        print("... will fetch from database, thank you for your patience ...")
        myDF = tab2download(tablename, classVar1, classKey1, classVar2, classKey2)
    return(myDF)

### ffcsv will be delivered as zip starting Feb 5th, 2024

In [18]:
from zipfile import ZipFile

# New flatfile csv (ffcsv) datastructure
* identical english variable lables in both en/de localizations
* Colon `;` delimited for both de/en
* Decimal point `.` in English but comma `,` in German
* utf-8 via API (as always) and utf-8 with BOM via web
* currently not sorted by regional key (may change in the future)
* only one `value` column
* counts and percentages mixed in this value
* observe `value_unit` to see which is which
* see `value_q` for how to interpret the data (see explanation of symbols above)
* ffcsv always comes zip-compressed with matching filename of .zip and .csv

In [19]:
def tab2df(ffcsv):
    zip_file = ZipFile(ffcsv)
    # name of internal csv file without classVar/classKey
    zcsv = ffcsv.split(myDownloads)[1][:18]+".csv"
    # decimal setting for german default
    df = pd.read_csv(zip_file.open(zcsv), delimiter = ';', decimal = ",",
                     # quality indicators (strings) may replace numerical values
                     na_values = ["...",".","-","/","x"],
                     # regional key (ARS, AGS) has leading zeroes, force as string
                     dtype = {"1_variable_attribute_code": str})

    print(ffcsv, "is ready to use")
    return(df)

### Explanation of symbols
`e`   final value  
`-`   Exactly zero or adjusted to zero  
`()`  Limited information value because the numerical value may have been modified relatively strongly by the confidentiality procedure
#### for Sample Data
`/`   No data because the numerical value is not sufficiently reliable  
`-`   no figures

## Example Table "5000H-2005" 
#### Haushalte: Größe des privaten Haushalts - Ausstattung der Wohnung/Fläche der Wohnung (20 m²-Intervalle)/Räume
Notice the slashes: On the second axis `/` denote variable features of which the first is selected by default  
Maybe experiment with all the options on the web first, switch on 'code' wherever possible, then come back here  
https://ergebnisse2011.zensus2022.de/datenbank/online/statistic/5000H/table/5000H-2005

In [24]:
# download single table, deliberate typo, table doesn't exist
myTable = tab2reuse("5000H-2905")

... will fetch from database, thank you for your patience ...
2024-02-06 08:59:03 : 5000H-2905 : {'Code': 2, 'Content': 'Ein Fehler ist aufgetreten.', 'Type': 'Fehler'}


In [75]:
# try again, now with correct name
myTable = tab2reuse("5000H-2005")

download/5000H-2005_de_flat____.zip is ready to use


#### Switching to "Fläche der Wohnung (20 m²-Intervalle)" instead of the default "Ausstattung der Wohnung"

In [13]:
# variable features switched
myTable = tab2reuse("5000H-2005", "HSHGR2", "", "WHGFL3", "")
# as of May 23rd, 2024 "" or * for classKey are selecting all keys as expected

... will fetch from database, thank you for your patience ...
2024-02-05 22:20:14 5000H-2005_de_flat download complete
download/5000H-2005_de_flat_HSHGR2__WHGFL3_.zip is ready to use


In [22]:
# asterisk notation, gets converted to x in filenames
myTable = tab2reuse("5000H-2005", "HSHGR2", "*", "WHGRM2", "*")

... will fetch from database, thank you for your patience ...
2024-02-06 08:53:19 5000H-2005_de_flat download complete
download/5000H-2005_de_flat_HSHGR2_x_WHGRM2_x.zip is ready to use


In [19]:
myTable#["2_variable_code"].value_counts()

,statistics_code,statistics_label,time_code,time_label,time,1_variable_code,1_variable_label,1_variable_attribute_code,1_variable_attribute_label,2_variable_code,...,2_variable_attribute_label,3_variable_code,3_variable_label,3_variable_attribute_code,3_variable_attribute_label,value,value_unit,value_variable_code,value_variable_label,value_q
0,5000H,Haushalte,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,130625214065,Züsedom,HSHGR2,...,Insgesamt,WHGFL3,Fläche der Wohnung (20 m²-Intervalle),NaN,Insgesamt,120,Anzahl,HSH002,Haushalte,e
1,5000H,Haushalte,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,130535312003,Altkalen,HSHGR2,...,Insgesamt,WHGFL3,Fläche der Wohnung (20 m²-Intervalle),NaN,Insgesamt,337,Anzahl,HSH002,Haushalte,e
2,5000H,Haushalte,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,010615153056,Kremperheide,HSHGR2,...,Insgesamt,WHGFL3,Fläche der Wohnung (20 m²-Intervalle),NaN,Insgesamt,1037,Anzahl,HSH002,Haushalte,e
3,5000H,Haushalte,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,031515407033,Ummern,HSHGR2,...,Insgesamt,WHGFL3,Fläche der Wohnung (20 m²-Intervalle),NaN,Insgesamt,588,Anzahl,HSH002,Haushalte,e
4,5000H,Haushalte,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,033560005005,Lilienthal,HSHGR2,...,Insgesamt,WHGFL3,Fläche der Wohnung (20 m²-Intervalle),NaN,Insgesamt,7875,Anzahl,HSH002,Haushalte,e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2944,5000H,Haushalte,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,130560072072,"Waren (Müritz), Stadt",HSHGR2,...,Insgesamt,WHGFL3,Fläche der Wohnung (20 m²-Intervalle),NaN,Insgesamt,10695,Anzahl,HSH002,Haushalte,e
2945,5000H,Haushalte,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,010575747010,Boksee,HSHGR2,...,Insgesamt,WHGFL3,Fläche der Wohnung (20 m²-Intervalle),NaN,Insgesamt,181,Anzahl,HSH002,Haushalte,e
2946,5000H,Haushalte,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,130605023093,Gallin-Kuppentin,HSHGR2,...,Insgesamt,WHGFL3,Fläche der Wohnung (20 m²-Intervalle),NaN,Insgesamt,244,Anzahl,HSH002,Haushalte,e
2947,5000H,Haushalte,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,130525222004,Bartow,HSHGR2,...,Insgesamt,WHGFL3,Fläche der Wohnung (20 m²-Intervalle),NaN,Insgesamt,240,Anzahl,HSH002,Haushalte,e


#### Use classifying variable/key pairs to refine the table, e.g. Single Households only that occupy the largest floor space 

In [25]:
# pruned table for a specific research question
myTable = tab2reuse("5000H-2005", "HSHGR2", "PERSON01", "WHGFL3", "WFL200BXXX")

download/5000H-2005_de_flat_HSHGR2_PERSON01_WHGFL3_WFL200BXXX.zip is ready to use


In [26]:
myTable

,statistics_code,statistics_label,time_code,time_label,time,1_variable_code,1_variable_label,1_variable_attribute_code,1_variable_attribute_label,2_variable_code,...,2_variable_attribute_label,3_variable_code,3_variable_label,3_variable_attribute_code,3_variable_attribute_label,value,value_unit,value_variable_code,value_variable_label,value_q
0,5000H,Haushalte,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,130625214064,Zerrenthin,HSHGR2,...,1 Person,WHGFL3,Fläche der Wohnung (20 m²-Intervalle),NaN,Insgesamt,48.0,Anzahl,HSH002,Haushalte,e
1,5000H,Haushalte,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,010515163012,Buchholz (Kreis Dithmarschen),HSHGR2,...,1 Person,WHGFL3,Fläche der Wohnung (20 m²-Intervalle),WFL200BXXX,200 m² und mehr,3.0,Anzahl,HSH002,Haushalte,e
2,5000H,Haushalte,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,010615168021,Christinenthal,HSHGR2,...,Insgesamt,WHGFL3,Fläche der Wohnung (20 m²-Intervalle),WFL200BXXX,200 m² und mehr,3.0,Anzahl,HSH002,Haushalte,e
3,5000H,Haushalte,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,130625214065,Züsedom,HSHGR2,...,Insgesamt,WHGFL3,Fläche der Wohnung (20 m²-Intervalle),NaN,Insgesamt,120.0,Anzahl,HSH002,Haushalte,e
4,5000H,Haushalte,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,033595404014,Düdenbüttel,HSHGR2,...,Insgesamt,WHGFL3,Fläche der Wohnung (20 m²-Intervalle),WFL200BXXX,200 m² und mehr,24.0,Anzahl,HSH002,Haushalte,e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11791,5000H,Haushalte,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,010545489017,Braderup,HSHGR2,...,Insgesamt,WHGFL3,Fläche der Wohnung (20 m²-Intervalle),WFL200BXXX,200 m² und mehr,32.0,Anzahl,HSH002,Haushalte,e
11792,5000H,Haushalte,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,130585828069,Metelsdorf,HSHGR2,...,Insgesamt,WHGFL3,Fläche der Wohnung (20 m²-Intervalle),WFL200BXXX,200 m² und mehr,6.0,Anzahl,HSH002,Haushalte,e
11793,5000H,Haushalte,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,032565409013,Hilgermissen,HSHGR2,...,1 Person,WHGFL3,Fläche der Wohnung (20 m²-Intervalle),WFL200BXXX,200 m² und mehr,7.0,Anzahl,HSH002,Haushalte,e
11794,5000H,Haushalte,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,034560025025,Wietmarschen,HSHGR2,...,Insgesamt,WHGFL3,Fläche der Wohnung (20 m²-Intervalle),WFL200BXXX,200 m² und mehr,392.0,Anzahl,HSH002,Haushalte,e


#### Example Table with Sample Data: 2000S-2029
Personen: Höchster beruflicher Abschluss (ausführlich) - Art der Wohnungsnutzung/Gebäudetyp (Bauweise)/Gebäudetyp (Größe)  
https://ergebnisse2011.zensus2022.de/datenbank/online/table/2000S-2029

In [112]:
# Personen "Ohne oder noch kein Schulabschluss", die in freistehenden Einfamilienhäusern wohnen
myTable = tab2reuse("2000S-2029", "BILBA1", "ABSCH-X", "GEBTP2", "GEB-EIN-FREI")

2000S-2029_de_flat.csv
download/2000S-2029_de_flat_BILBA1_ABSCH-X_GEBTP2_GEB-EIN-FREI.zip is ready to use


In [113]:
myTable

,statistics_code,statistics_label,time_code,time_label,time,1_variable_code,1_variable_label,1_variable_attribute_code,1_variable_attribute_label,2_variable_code,...,2_variable_attribute_label,3_variable_code,3_variable_label,3_variable_attribute_code,3_variable_attribute_label,value,value_unit,value_variable_code,value_variable_label,value_q
0,2000S,"Bevölkerung: Bildung, Erwerb, Migration",STAG,Stichtag,2011-05-09,GEOGM3,Gemeinden mit mindestens 10 000 Einwohnern,010560041041,"Quickborn, Stadt",BILBA1,...,Ohne Abschluss,GEBTP2,Gebäudetyp (Größe),GEB-EIN-FREI,Freistehendes Einfamilienhaus,1500.0,Anzahl,PRS009,Personen ab 15 Jahren,e
1,2000S,"Bevölkerung: Bildung, Erwerb, Migration",STAG,Stichtag,2011-05-09,GEOGM3,Gemeinden mit mindestens 10 000 Einwohnern,010560050050,"Wedel, Stadt",BILBA1,...,Ohne Abschluss,GEBTP2,Gebäudetyp (Größe),NaN,Insgesamt,6450.0,Anzahl,PRS009,Personen ab 15 Jahren,e
2,2000S,"Bevölkerung: Bildung, Erwerb, Migration",STAG,Stichtag,2011-05-09,GEOGM3,Gemeinden mit mindestens 10 000 Einwohnern,034590012012,Bissendorf,BILBA1,...,Ohne Abschluss,GEBTP2,Gebäudetyp (Größe),GEB-EIN-FREI,Freistehendes Einfamilienhaus,1070.0,Anzahl,PRS009,Personen ab 15 Jahren,e
3,2000S,"Bevölkerung: Bildung, Erwerb, Migration",STAG,Stichtag,2011-05-09,GEOGM3,Gemeinden mit mindestens 10 000 Einwohnern,031020000000,"Salzgitter, Stadt",BILBA1,...,Insgesamt,GEBTP2,Gebäudetyp (Größe),NaN,Insgesamt,83710.0,Anzahl,PRS009,Personen ab 15 Jahren,e
4,2000S,"Bevölkerung: Bildung, Erwerb, Migration",STAG,Stichtag,2011-05-09,GEOGM3,Gemeinden mit mindestens 10 000 Einwohnern,034610007007,"Nordenham, Stadt",BILBA1,...,Ohne Abschluss,GEBTP2,Gebäudetyp (Größe),GEB-EIN-FREI,Freistehendes Einfamilienhaus,2310.0,Anzahl,PRS009,Personen ab 15 Jahren,e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1131,2000S,"Bevölkerung: Bildung, Erwerb, Migration",STAG,Stichtag,2011-05-09,GEOGM3,Gemeinden mit mindestens 10 000 Einwohnern,034550026026,"Varel, Stadt",BILBA1,...,Ohne Abschluss,GEBTP2,Gebäudetyp (Größe),NaN,Insgesamt,6090.0,Anzahl,PRS009,Personen ab 15 Jahren,e
1132,2000S,"Bevölkerung: Bildung, Erwerb, Migration",STAG,Stichtag,2011-05-09,GEOGM3,Gemeinden mit mindestens 10 000 Einwohnern,034590033033,Wallenhorst,BILBA1,...,Insgesamt,GEBTP2,Gebäudetyp (Größe),GEB-EIN-FREI,Freistehendes Einfamilienhaus,6780.0,Anzahl,PRS009,Personen ab 15 Jahren,e
1133,2000S,"Bevölkerung: Bildung, Erwerb, Migration",STAG,Stichtag,2011-05-09,GEOGM3,Gemeinden mit mindestens 10 000 Einwohnern,130050000000,"Stralsund, Hansestadt",BILBA1,...,Insgesamt,GEBTP2,Gebäudetyp (Größe),GEB-EIN-FREI,Freistehendes Einfamilienhaus,4660.0,Anzahl,PRS009,Personen ab 15 Jahren,e
1134,2000S,"Bevölkerung: Bildung, Erwerb, Migration",STAG,Stichtag,2011-05-09,GEOGM3,Gemeinden mit mindestens 10 000 Einwohnern,130550050050,"Neustrelitz, Stadt",BILBA1,...,Insgesamt,GEBTP2,Gebäudetyp (Größe),GEB-EIN-FREI,Freistehendes Einfamilienhaus,2220.0,Anzahl,PRS009,Personen ab 15 Jahren,e


#### Example Table with Percentages: 1000A-1009
Personen: ...  
https://ergebnisse2011.zensus2022.de/datenbank/online/table/1000A-1009

In [118]:
# Personen
myTable = tab2reuse("1000A-1009", "FAMST1", "VERWITWET")

1000A-1009_de_flat.csv
download/1000A-1009_de_flat_FAMST1_VERWITWET__.zip is ready to use


In [119]:
myTable

,statistics_code,statistics_label,time_code,time_label,time,1_variable_code,1_variable_label,1_variable_attribute_code,1_variable_attribute_label,2_variable_code,2_variable_label,2_variable_attribute_code,2_variable_attribute_label,value,value_unit,value_variable_code,value_variable_label,value_q
0,1000A,Bevölkerung kompakt,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,010585864147,Schülp b. Nortorf,FAMST1,Familienstand (ausführlich),VERWITWET,Verwitwet,6.6,%,PRS018,Personen,e
1,1000A,Bevölkerung kompakt,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,010585864147,Schülp b. Nortorf,FAMST1,Familienstand (ausführlich),VERWITWET,Verwitwet,52.0,Anzahl,PRS018,Personen,e
2,1000A,Bevölkerung kompakt,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,031545403025,Warberg,FAMST1,Familienstand (ausführlich),NaN,Insgesamt,100.0,%,PRS018,Personen,e
3,1000A,Bevölkerung kompakt,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,031545403025,Warberg,FAMST1,Familienstand (ausführlich),NaN,Insgesamt,890.0,Anzahl,PRS018,Personen,e
4,1000A,Bevölkerung kompakt,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,010595990109,Esgrus,FAMST1,Familienstand (ausführlich),NaN,Insgesamt,100.0,%,PRS018,Personen,e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11795,1000A,Bevölkerung kompakt,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,033515404018,Nienhagen (Landkreis Celle),FAMST1,Familienstand (ausführlich),VERWITWET,Verwitwet,423.0,Anzahl,PRS018,Personen,e
11796,1000A,Bevölkerung kompakt,STAG,Stichtag,2011-05-09,GEODL1,Deutschland,DG,Deutschland,FAMST1,Familienstand (ausführlich),NaN,Insgesamt,100.0,%,PRS001,Personen,e
11797,1000A,Bevölkerung kompakt,STAG,Stichtag,2011-05-09,GEODL1,Deutschland,DG,Deutschland,FAMST1,Familienstand (ausführlich),NaN,Insgesamt,80219695.0,Anzahl,PRS001,Personen,e
11798,1000A,Bevölkerung kompakt,STAG,Stichtag,2011-05-09,GEODL1,Deutschland,DG,Deutschland,FAMST1,Familienstand (ausführlich),VERWITWET,Verwitwet,7.1,%,PRS001,Personen,e


In [121]:
# Fläche qkm
myTable = tab2reuse("1000A-0001")

download/1000A-0001_de_flat____.zip is ready to use


In [122]:
myTable

,statistics_code,statistics_label,time_code,time_label,time,1_variable_code,1_variable_label,1_variable_attribute_code,1_variable_attribute_label,value,value_unit,value_variable_code,value_variable_label,value_q
0,1000A,Bevölkerung kompakt,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,033545403021,"Schnackenburg, Stadt",25.00,Ew/qkm,PRS017,Bevölkerungsdichte,e
1,1000A,Bevölkerung kompakt,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,033545403021,"Schnackenburg, Stadt",23.70,qkm,FLC001,Fläche,e
2,1000A,Bevölkerung kompakt,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,033545403021,"Schnackenburg, Stadt",592.00,Anzahl,PRS018,Personen,e
3,1000A,Bevölkerung kompakt,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,010545489109,Risum-Lindholm,100.00,Ew/qkm,PRS017,Bevölkerungsdichte,e
4,1000A,Bevölkerung kompakt,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,010545489109,Risum-Lindholm,36.03,qkm,FLC001,Fläche,e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8842,1000A,Bevölkerung kompakt,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,010585864011,Bargstedt (Kreis Rendsburg-Eckernförde),25.32,qkm,FLC001,Fläche,e
8843,1000A,Bevölkerung kompakt,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,010585864011,Bargstedt (Kreis Rendsburg-Eckernförde),729.00,Anzahl,PRS018,Personen,e
8844,1000A,Bevölkerung kompakt,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,033575405050,Vierden,37.00,Ew/qkm,PRS017,Bevölkerungsdichte,e
8845,1000A,Bevölkerung kompakt,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,033575405050,Vierden,21.30,qkm,FLC001,Fläche,e


In [123]:
# scratch
myTable = tab2reuse("1000A-1001")

... will fetch from database, thank you for your patience ...
2024-02-05 19:10:38 1000A-1001_de_flat download complete
download/1000A-1001_de_flat____.zip is ready to use


In [124]:
myTable

,statistics_code,statistics_label,time_code,time_label,time,1_variable_code,1_variable_label,1_variable_attribute_code,1_variable_attribute_label,2_variable_code,2_variable_label,2_variable_attribute_code,2_variable_attribute_label,value,value_unit,value_variable_code,value_variable_label,value_q
0,1000A,Bevölkerung kompakt,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,010600039039,Henstedt-Ulzburg,ALTGR2,Alter (10er-Jahresgruppen),ALT020B029,20 bis 29 Jahre,9.3,%,PRS018,Personen,e
1,1000A,Bevölkerung kompakt,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,010600039039,Henstedt-Ulzburg,ALTGR2,Alter (10er-Jahresgruppen),ALT020B029,20 bis 29 Jahre,2472.0,Anzahl,PRS018,Personen,e
2,1000A,Bevölkerung kompakt,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,034625402001,Blomberg,ALTGR2,Alter (10er-Jahresgruppen),NaN,Insgesamt,100.0,%,PRS018,Personen,e
3,1000A,Bevölkerung kompakt,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,034625402001,Blomberg,ALTGR2,Alter (10er-Jahresgruppen),NaN,Insgesamt,1656.0,Anzahl,PRS018,Personen,e
4,1000A,Bevölkerung kompakt,STAG,Stichtag,2011-05-09,GEOGM1,Gemeinden,032410012012,"Neustadt am Rübenberge, Stadt",ALTGR2,Alter (10er-Jahresgruppen),ALT030B039,30 bis 39 Jahre,10.4,%,PRS018,Personen,e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58995,1000A,Bevölkerung kompakt,STAG,Stichtag,2011-05-09,GEODL1,Deutschland,DG,Deutschland,ALTGR2,Alter (10er-Jahresgruppen),ALT080BXXX,80 Jahre und älter,4213248.0,Anzahl,PRS001,Personen,e
58996,1000A,Bevölkerung kompakt,STAG,Stichtag,2011-05-09,GEODL1,Deutschland,DG,Deutschland,ALTGR2,Alter (10er-Jahresgruppen),ALT020B029,20 bis 29 Jahre,12.1,%,PRS001,Personen,e
58997,1000A,Bevölkerung kompakt,STAG,Stichtag,2011-05-09,GEODL1,Deutschland,DG,Deutschland,ALTGR2,Alter (10er-Jahresgruppen),ALT020B029,20 bis 29 Jahre,9708174.0,Anzahl,PRS001,Personen,e
58998,1000A,Bevölkerung kompakt,STAG,Stichtag,2011-05-09,GEODL1,Deutschland,DG,Deutschland,ALTGR2,Alter (10er-Jahresgruppen),ALT050B059,50 bis 59 Jahre,14.5,%,PRS001,Personen,e
